In [2]:
import os   
import pandas as pd
import numpy as np


## Extract from log functions

In [3]:
def extract_dict_from_files(log_dir):
    all_files = os.listdir(log_dir)
    bigdict = {}
    for file in all_files:
        taskL = file.split('=')[1][:-6].split('_')
        task = taskL[0]
        coeff = file.split('=')[2][:-5]
        if len(taskL)>1:
            coeff+=taskL[1]
        if task not in bigdict.keys():
            bigdict[task] = {}
        if coeff not in bigdict[task].keys():
            bigdict[task][coeff] = {'Accuracy':[], 'Attention':[]}

        f = open(log_dir+file, "r")
        for i, line in enumerate(f):
            if len(line)>2:
                if line[0] == 'F':
                    line = line.split()
                    bigdict[task][coeff][line[2]].append(float(line[-1]))
    return bigdict


def pretty(d, indent=0):
    for key, value in d.items():
        print('\t' * indent + str(key))
        if isinstance(value, dict):
            pretty(value, indent+1)
        else:
            print('\t' * (indent+1) + str(value))
            
def avg(d, indent=0):
    return_dict = {}
    for key, value in d.items():
        if key not in return_dict.keys():
            return_dict[key] = {}
        for key1, value1 in value.items():
            if key1 not in return_dict.keys():
                return_dict[key][key1] = {}
            for key2, value2 in value1.items():
#                 return_dict[key][key1]['seeds'] = len(value2)
                return_dict[key][key1][key2] = str(round(np.mean(value2), 2))+u"\u00B1"+str(round(np.std(value2), 2))
    return return_dict

def df(d):
    keys = []
    dataframes = []
    for key, value in d.items():
        keys.append(key)
        dataframes.append(pd.DataFrame.from_dict(value).T)
        
    df = pd.concat(dataframes, keys=keys,  axis=1)

    
#     df.columns.set_levels(['seeds', 'Acc.', 'A.M.'],level=1,inplace=True)

    df.columns.set_levels(['Acc.', 'A.M.'],level=1,inplace=True)

    df.columns.set_levels(['Sequence Reverse', 'Sequence Copy', 'En → De MT', 'Bigram Flip'],level=0,inplace=True)

    df = df.rename(index={'0.1':"Manipulated0.1", '0.0no-attn': "None", '0.0uniform':"Uniform", '1.0':"Manipulated1.0", '0.0':"Dot-Product"})
    df = df.reindex(['Dot-Product','Uniform', 'None','Manipulated0.1', 'Manipulated1.0'])
    df.index.name = 'Attention'
    
    df = df.reindex(['Bigram Flip', 'Sequence Copy', 'Sequence Reverse', 'En → De MT'], axis="columns", level=0)
    lambdaa = [0.0, 0.0, 0.0, 0.1, 1.0]
    df.insert(0, '\u03BB', lambdaa, True)
    return df



## Extract from logs
Change folder dir if logs are in different directory. 

In [5]:
folder_dir = ""
bigdict = extract_dict_from_files(folder_dir+"logs/")

# pretty(bigdict)

avg_dict = avg(bigdict)
df  = df(avg_dict)
df


λ Bigram Flip             Sequence Copy              \
                           Acc.        A.M.          Acc.        A.M.   
Attention                                                               
Dot-Product     0.0   100.0±0.0  93.89±0.16     100.0±0.0  94.11±0.12   
Uniform         0.0  92.49±2.52    4.71±0.0    81.76±0.77    4.73±0.0   
None            0.0  94.89±1.39     0.0±0.0     83.79±3.7     0.0±0.0   
Manipulated0.1  0.1   100.0±0.0  18.72±9.49     100.0±0.0  10.81±6.52   
Manipulated1.0  1.0  99.91±0.04   0.01±0.01    99.89±0.06   0.02±0.01   

               Sequence Reverse               En → De MT              
                           Acc.         A.M.        Acc.        A.M.  
Attention                                                             
Dot-Product          99.99±0.01   93.49±0.95  37.58±0.72  24.85±1.82  
Uniform              80.91±4.16     4.74±0.0  32.35±0.91    5.96±0.0  
None                  86.61±5.7      0.0±0.0  30.63±0.68     0.0±0.0  
Manipulated0.1        100.0±0.0  17.44±11.53  36.75±0.84  16.53±4.07  
Manipulated1.0       99.87±0.07    0.02±0.01  33.22±1.15   1.19±0.73

## Test run
### 1 seed and 1 epoch

In [6]:
import subprocess


subprocess.check_call('CUDA_VISIBLE_DEVICES=0 bash test_tasks.sh', shell=True)

subprocess.check_call('CUDA_VISIBLE_DEVICES=0 bash test_uniform_no_attn_baselines.sh', shell=True)

0

## Full run


In [ ]:

subprocess.check_call('CUDA_VISIBLE_DEVICES=0 bash run_tasks.sh', shell=True)

subprocess.check_call('CUDA_VISIBLE_DEVICES=0 bash run_uniform_no_attn_baselines.sh', shell=True)